<a href="https://colab.research.google.com/github/adeepH/SDPRA-2021-SharedTask/blob/main/Notebooks/Inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/adeepH/SPDRA-2021-SharedTask.git 

Cloning into 'SPDRA-2021-SharedTask'...
remote: Enumerating objects: 102, done.
remote: Counting objects: 100% (102/102), done.
remote: Compressing objects: 100% (93/93), done.
remote: Total 102 (delta 46), reused 6 (delta 2), pack-reused 0
Receiving objects: 100% (102/102), 4.46 MiB | 15.02 MiB/s, done.
Resolving deltas: 100% (46/46), done.


In [2]:
cd /content/SPDRA-2021-SharedTask/

/content/SPDRA-2021-SharedTask


In [3]:
!pip install -r requirements.txt

     |████████████████████████████████| 1.1MB 11.3MB/s 
     |████████████████████████████████| 1.1MB 51.3MB/s 
     |████████████████████████████████| 3.0MB 50.2MB/s 
     |████████████████████████████████| 890kB 49.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=7b9c454a61551ce5d06e653e6a3350006ef2194ebcbe534c3b3ddd3869305f7d
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [4]:
cd  

/root


In [5]:
cd /content/SPDRA-2021-SharedTask/scripts/

/content/SPDRA-2021-SharedTask/scripts


In [6]:
import pandas as pd
import numpy as np
import torch
from dataset import *
from model import *
from utils import *

In [7]:
tokenizer = AutoTokenizer.from_pretrained('roberta-base') 
device = 'cuda'
MAX_LEN = 128
BATCH_SIZE = 32

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
model = RFClassifier(7,'roberta-base')
model.load_state_dict(torch.load('/content/drive/MyDrive/spdra2021/saved_models/roberta.bin'))
model.to(device)
for param in model.parameters():
    param.requires_grad = False
model.eval()

RFClassifier(
  (auto): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,),

In [10]:
test = pd.read_csv('/content/drive/MyDrive/spdra2021/Datasets/test.csv',delimiter=',',
                 header=None,names=['text']) 

In [ ]:
!python dataset.py

In [11]:

class RFDataset(Dataset):
  def __init__(self,text,tokenizer,max_len):
    self.text = text
    #self.label = label
    self.tokenizer = tokenizer
    self.max_len = max_len
  
  def __len__(self):
    return len(self.text)
  
  def __getitem__(self,item):
    text = str(self.text[item])
    #label = self.label[item]

    encoding = self.tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length = self.max_len,
        return_token_type_ids = False,
        padding = 'max_length',
        return_attention_mask= True,
        return_tensors='pt',
        truncation=True
    )

    return {
        'text' : text,
        'input_ids' : encoding['input_ids'].flatten(),
        'attention_mask' : encoding['attention_mask'].flatten(),
        #'label' : torch.tensor(label,dtype=torch.long)

    }


In [12]:
def create_dataloader(df,tokenizer,max_len,batch_size):
  ds = RFDataset(
      text = df.text.to_numpy(),
      #label = df.label.to_numpy(),
      tokenizer = tokenizer,
      max_len = max_len
  )

  return DataLoader(ds,
                    batch_size = batch_size,
                    shuffle = False,
                    num_workers=4)


In [13]:
test_data_loader = create_dataloader(test,tokenizer,MAX_LEN,BATCH_SIZE)

In [14]:
def get_predictions(model, data_loader):
  model = model.eval()
  sentence = []
  predictions = []
  prediction_probs = []
  #real_values = []
  with torch.no_grad():
    for d in data_loader:
      texts = d["text"]
      input_ids = d["input_ids"].to(device)
      attention_mask = d["attention_mask"].to(device)
      #labels = d["label"].to(device)
      outputs = model(
        input_ids=input_ids,
        attention_mask=attention_mask
      )
      _, preds = torch.max(outputs, dim=1)
      probs = nn.functional.softmax(outputs,dim=1)
      sentence.extend(texts)
      predictions.extend(preds)
      prediction_probs.extend(probs)
      #real_values.extend(labels)
  predictions = torch.stack(predictions).cpu().detach().numpy().tolist()
  prediction_probs = torch.stack(prediction_probs).cpu().detach().numpy().tolist()
  #real_values = torch.stack(real_values).cpu().detach().numpy().tolist()
  return sentence, predictions, prediction_probs #, real_values

In [15]:
y_review_texts, y_pred, y_pred_probs = get_predictions(
  model,
  test_data_loader
)

In [16]:
 class_name = ['CL' ,'CR' ,'DC' ,'DS' ,'LO' , 'NI' , 'SE' ]
k = pd.DataFrame(y_pred_probs,columns=class_name)
k['result'] = k.idxmax(axis = 1) 
k['result'] = k['result'].apply({'CL':0,'CR':1,'DC':2,
'DS':3,'LO':4, 'NI':5, 'SE':6}.get) 
k['abstract'] = y_review_texts

In [17]:
k

,CL,CR,DC,DS,LO,NI,SE,result,abstract
0,0.002313,0.002387,0.980938,0.003960,0.001252,0.005076,0.004074,2,This paper analyses the possibilities of per...
1,0.002194,0.003206,0.978623,0.005403,0.000872,0.007745,0.001957,2,A finite element method is presented to comp...
2,0.997938,0.000090,0.000247,0.000461,0.000620,0.000309,0.000335,0,This paper includes a reflection on the role...
3,0.006236,0.298282,0.384594,0.064093,0.015241,0.203037,0.028518,2,"In this document, we describe the fractal st..."
4,0.000752,0.000967,0.001257,0.994632,0.001538,0.000534,0.000321,3,We show how to test whether a graph with n v...
...,...,...,...,...,...,...,...,...,...
6995,0.002025,0.000951,0.001780,0.002676,0.989488,0.000525,0.002555,4,It is common practice to compare the computa...
6996,0.002715,0.000618,0.000790,0.001602,0.991278,0.000459,0.002538,4,Defeasible reasoning is a simple but efficie...
6997,0.001672,0.001141,0.001263,0.002764,0.989630,0.000520,0.003011,4,The almost periodic functions form a natural...
6998,0.002069,0.000676,0.000909,0.001749,0.991237,0.000449,0.002912,4,A notion of alternating timed automata is pr...


In [26]:
k

,CL,CR,DC,DS,LO,NI,SE,result,abstract
0,0.002313,0.002387,0.980938,0.003960,0.001252,0.005076,0.004074,2,This paper analyses the possibilities of per...
1,0.002194,0.003206,0.978623,0.005403,0.000872,0.007745,0.001957,2,A finite element method is presented to comp...
2,0.997938,0.000090,0.000247,0.000461,0.000620,0.000309,0.000335,0,This paper includes a reflection on the role...
3,0.006236,0.298282,0.384594,0.064093,0.015241,0.203037,0.028518,2,"In this document, we describe the fractal st..."
4,0.000752,0.000967,0.001257,0.994632,0.001538,0.000534,0.000321,3,We show how to test whether a graph with n v...
...,...,...,...,...,...,...,...,...,...
6995,0.002025,0.000951,0.001780,0.002676,0.989488,0.000525,0.002555,4,It is common practice to compare the computa...
6996,0.002715,0.000618,0.000790,0.001602,0.991278,0.000459,0.002538,4,Defeasible reasoning is a simple but efficie...
6997,0.001672,0.001141,0.001263,0.002764,0.989630,0.000520,0.003011,4,The almost periodic functions form a natural...
6998,0.002069,0.000676,0.000909,0.001749,0.991237,0.000449,0.002912,4,A notion of alternating timed automata is pr...


In [18]:
k.result.value_counts()

5    1203
0    1192
1    1130
3    1087
6     860
2     794
4     734
Name: result, dtype: int64

In [19]:
from sklearn.metrics import classification_report,confusion_matrix
print(classification_report(y_pred,k['result'],target_names=class_name,zero_division=0))

              precision    recall  f1-score   support

          CL       1.00      1.00      1.00      1192
          CR       1.00      1.00      1.00      1130
          DC       1.00      1.00      1.00       794
          DS       1.00      1.00      1.00      1087
          LO       1.00      1.00      1.00       734
          NI       1.00      1.00      1.00      1203
          SE       1.00      1.00      1.00       860

    accuracy                           1.00      7000
   macro avg       1.00      1.00      1.00      7000
weighted avg       1.00      1.00      1.00      7000



In [20]:
confusion_matrix(y_pred,k['result'])

array([[1192,    0,    0,    0,    0,    0,    0],
       [   0, 1130,    0,    0,    0,    0,    0],
       [   0,    0,  794,    0,    0,    0,    0],
       [   0,    0,    0, 1087,    0,    0,    0],
       [   0,    0,    0,    0,  734,    0,    0],
       [   0,    0,    0,    0,    0, 1203,    0],
       [   0,    0,    0,    0,    0,    0,  860]])

In [21]:
cd /content/drive/MyDrive/spdra2021/pred_probs/

/content/drive/MyDrive/spdra2021/pred_probs


In [22]:
k.to_csv('roberta.csv')

In [28]:
k['result'] = k['result'].apply({0:'CL', 1:'CR', 2:'DC',
3:'DS', 4:'LO', 5:'NI', 6:'SE' }.get)

In [29]:
result = k['result'].to_numpy()
print(len(result))
np.savetxt("scibert.txt", result, fmt = "%s")

7000


In [30]:
from google.colab import files
files.download('scibert.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>